In [1]:
import pandas as pd
import sklearn

In [2]:
df = pd.read_csv('LA NO2.csv', parse_dates=['date','utc','local'], index_col='utc')

In [3]:
df.head()

,parameter,location,value,concentration_units,city,country,sourcename,sourcetype,mobile,latitude,...,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth,weather_events
utc,,,,,,,,,,,,,,,,,,,,,
2020-01-02 18:00:00,no2,North Holywood,0.005,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 22:00:00,no2,North Holywood,0.016,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 17:00:00,no2,North Holywood,0.005,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 23:00:00,no2,North Holywood,0.015,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 21:00:00,no2,North Holywood,0.010,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN


In [4]:
df.columns

Index(['parameter', 'location', 'value', 'concentration_units', 'city',
       'country', 'sourcename', 'sourcetype', 'mobile', 'latitude',
       'longitude', 'local', 'source', 'url', 'averaging_time_unit',
       'averaging_time', 'station', 'date', 'latitude-2', 'longitude-2',
       'elevation', 'name', 'temperature', 'dewpoint', 'slp', 'stp',
       'visibility', 'wind_speed', 'max_sustained_wind', 'gust', 'max_temp',
       'min_temp', 'precipitation', 'snow_depth', 'weather_events'],
      dtype='object')

In [5]:
df = df.resample('d', on='date').mean()
df = df.interpolate(method='time')
df.head()

,value,mobile,latitude,longitude,url,averaging_time,station,latitude-2,longitude-2,elevation,...,slp,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.010444,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1012.5,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0.0
2020-01-03,0.027417,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1019.9,994.0,10.0,1.6,6.0,NaN,73.0,45.0,0.0,0.0
2020-01-04,0.029000,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1025.1,999.1,10.0,1.1,6.0,NaN,73.0,42.1,0.0,0.0
2020-01-05,0.023833,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1024.2,998.0,10.0,2.8,11.1,18.100000,73.0,42.1,0.0,0.0
2020-01-06,0.024357,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1023.5,997.6,10.0,2.5,6.0,16.733333,75.0,42.1,0.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 0.11718023393643871
   Coefficients: [-9.22760766e-05 -2.02168957e-03 -1.96661765e-04  5.78065619e-04
 -2.83923619e-04 -3.61156836e-04 -1.90506008e-04]
   Score 0.688644219845947
MAE is 0.003475423761773458
RMSE is 0.004573000160813276
MSE is 2.0912330470798244e-05
R^2: 0.688644219845947


In [7]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

power = 8
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 441.9040667711327
   Coefficients: [-5.69032364e-13 -1.22041845e-13  4.76771172e-13 ... -4.30875705e-14
 -7.17679723e-13 -7.22006846e-13]
   Score 0.9923528722032307
MAE is 0.000543774767130865
RMSE is 0.0007166744542666714
MSE is 5.136222733984313e-07
R^2: 0.9923528722032307


In [8]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 8
MAE is 0.33522309637734843
RMSE is 1.441519508811445
MSE is 2.0779784942839896
R^2: -30681.4785408294


Model is overfit at degree 8

Results for linear regression on training data
Internal Parameters:
   Bias is -130.8516278519432
   Coefficients: [ 5.41883862e-05  4.42209879e-01 -1.09368613e+01  1.09884784e+01
 -1.43649845e+01 -6.09250644e+00  1.80941231e+01  3.77682937e+00
 -4.99080145e-04  2.09354086e-02 -2.14214424e-02  2.82006940e-02
  1.13039005e-02 -3.48095235e-02 -7.32139779e-03  2.57011331e-02
  1.95143256e-02 -1.65788384e-02  1.28729868e-02  2.30412017e-02
 -6.86676289e-03 -2.21762026e-02  2.97365789e-02  1.90361208e-02
 -5.33929568e-02 -4.55502541e-03 -1.22867089e-03  6.90261004e-04
  1.20465228e-02 -7.87453724e-05 -9.33306152e-03  2.33322058e-02
  4.55694551e-03 -2.53187798e-02 -5.35386085e-03 -1.50633818e-03
  1.87587625e-07 -9.97923214e-06  1.04398872e-05 -1.38375336e-05
 -5.20883493e-06  1.67205471e-05  3.54623127e-06 -2.55144630e-05
 -2.01242573e-05  1.79711969e-05 -1.21384618e-05 -2.55914197e-05
  6.57171836e-06  2.23652354e-05 -3.01009496e-05 -1.95375813e-05
  5.39470244e-05  4.48155497e-06  1.3873

In [12]:
power = 2
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 0.33815072081383407
   Coefficients: [-1.49101913e-12 -7.80528375e-04 -1.22697060e-01  8.26026250e-02
  6.39855776e-03 -9.76875423e-03 -4.34229226e-03  5.73264211e-03
  4.68264051e-07  1.18339266e-04 -8.22941595e-05 -5.02173456e-06
  8.63720100e-06  3.55138369e-06 -5.56745154e-06  1.20341228e-04
  4.35562312e-05 -5.84328222e-05 -1.78247372e-05  7.96814588e-05
  5.46740277e-05  2.55371978e-06 -1.17917321e-05 -2.62220322e-06
  5.27434400e-06 -2.57396618e-05  2.36397060e-05  1.00059566e-05
 -6.92021806e-05 -7.11311248e-06  4.66683268e-07  1.39972045e-05
 -1.61031714e-05  3.10456508e-05  1.93353857e-05 -2.56593373e-06]
   Score 0.7437630560387788
MAE is 0.003089545113004879
RMSE is 0.004148524157285349
MSE is 1.7210252683580114e-05
R^2: 0.7437630560387788


In [13]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 2
MAE is 0.0032595355907961156
RMSE is 0.004471836131248678
MSE is 1.9997318384741143e-05
R^2: 0.7047287573467547


In [10]:
power = 3
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

In [11]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 3
MAE is 0.003763551084645628
RMSE is 0.005212595151025
MSE is 2.7171148208489343e-05
R^2: 0.5988032724448717


In [14]:
power = 4
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is -1290.9751031639005
   Coefficients: [-1.21275579e-03 -1.67127782e+00 -1.75673924e-01  2.33976299e-01
  2.93147755e-01  1.07914156e-01  1.53972223e-01  1.05826574e-01
  1.48189348e-02 -5.75809742e-01  2.52655371e-01 -2.40277363e-01
  9.13208064e-02  2.42497588e-01 -1.28863583e-01  1.64925741e+01
  1.19356148e+01  3.15781799e+00  7.08362419e+00 -1.65221226e+00
 -4.67665461e+00 -1.14510162e+01  6.08909582e+00 -2.45577663e+00
 -7.53193141e+00  1.76728316e+00  2.23697515e+00 -9.75138411e-01
 -3.84859172e+00  2.27107390e+00  1.67552997e-01 -1.44730161e+00
  1.01076608e+00  2.25285013e+00 -1.06427970e+00 -3.85021663e-01
 -1.95007784e-05  1.15509466e-03 -5.01965951e-04  4.76654093e-04
 -1.86973260e-04 -4.77652263e-04  2.56598118e-04 -3.25439580e-02
 -2.47833640e-02 -6.23388991e-03 -1.35922451e-02  2.86087076e-03
  9.41474064e-03  2.30092823e-02 -1.18850164e-02  5.20855692e-03
  1.43313341e-02 -3.60584221e-03 -4.319

In [15]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 4
MAE is 0.006202615051260571
RMSE is 0.011897781783780953
MSE is 0.00014155721137446988
R^2: -1.090168936899131


In [16]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=1000, max_features = 'sqrt', max_depth =100,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')


MAE is 0.0011629946135910665
RMSE is 0.0016028450336462608
MSE is 2.569112201884483e-06
R^2: 0.9617494599639183


In [18]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.003390989694717383
RMSE is 0.004565226583856584
MSE is 2.0841293761950853e-05
R^2: 0.6922670035454247


In [19]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =50,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')


MAE is 0.001175434393798324
RMSE is 0.0016303570660204845
MSE is 2.6580641627229225e-06
R^2: 0.9604250878571481


In [20]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.003395065764083209
RMSE is 0.004565083149599456
MSE is 2.083998416275689e-05
R^2: 0.6922863404872055


In [21]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =25,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0011761363563272315
RMSE is 0.0016309217522757278
MSE is 2.6599057620461303e-06
R^2: 0.9603976689812457


In [22]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.0033979225265327088
RMSE is 0.004568196111398462
MSE is 2.0868415712196028e-05
R^2: 0.6918665332524586


In [23]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =10,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0016182614147895564
RMSE is 0.002092405524848461
MSE is 4.3781608804163635e-06
R^2: 0.9348152183007292


In [24]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.003389596349879812
RMSE is 0.004603565779571652
MSE is 2.1192817886843145e-05
R^2: 0.6870765593477296


In [25]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =5,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0028782179199543885
RMSE is 0.0038255518958585904
MSE is 1.4634847307907255e-05
R^2: 0.7821072927595634


In [26]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.0036191096269724676
RMSE is 0.0048610703296104694
MSE is 2.3630004749419237e-05
R^2: 0.6510901745912555


In [27]:
rfmodel = RandomForestRegressor(n_estimators=10, max_features = 'sqrt', max_depth =5,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.002948581190286136
RMSE is 0.003909881647397569
MSE is 1.5287174496656333e-05
R^2: 0.7723950399309136


In [28]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
MAE is 0.0036372464629853027
RMSE is 0.004916573249658079
MSE is 2.4172692519253404e-05
R^2: 0.6430770955829259


In [29]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(100,400,100), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.006797221772488456
RMSE is 0.00909196462590496
MSE is 8.266382075870712e-05
R^2: -0.23075036705173146


In [30]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for linear regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
MAE is 0.007075138564431895
RMSE is 0.00931958637475207
MSE is 8.68546901964644e-05
R^2: -0.282456567983711


In [31]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(25,50,25), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.01988643199600008
RMSE is 0.026082069772010785
MSE is 0.0006802743635920388
R^2: -9.128347746357901


In [32]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for linear regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
MAE is 0.019089210198473847
RMSE is 0.02560459519676495
MSE is 0.0006555952951901987
R^2: -8.680219805678561


In [33]:
X = train_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is 0.03281751347715747
   Coefficients: [-8.31826904e-06 -2.63873659e-03  1.67956031e-04 -2.87164039e-04]
   Score 0.6391266234261839
MAE is 0.0036774340702419262
RMSE is 0.004923230528721069
MSE is 2.423819883893114e-05
R^2: 0.6391266234261839


In [34]:
X = train_set[['wind_speed']]
y = train_set['value']

X_test = test_set[['wind_speed']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
    Default Settings
Internal Parameters:
   Bias is 0.02697364742425181
   Coefficients: [-0.0029801]
   Score 0.4659905938757001
MAE is 0.004674262749685398
RMSE is 0.005988901962163799
MSE is 3.58669467124094e-05
R^2: 0.4659905938757001


In [35]:
X = train_set[['max_sustained_wind']]
y = train_set['value']

X_test = test_set[['max_sustained_wind']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 0.024799405583871073
   Coefficients: [-0.00110766]
   Score 0.18614956963949847
MAE is 0.0061137644573721365
RMSE is 0.007393414401126418
MSE is 5.466257650678351e-05
R^2: 0.18614956963949847


In [36]:
power = 2
poly_process = PolynomialFeatures(degree = power, include_bias=True)

X = train_set[['stp', 'precipitation' ,'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'precipitation', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is -0.7730912047919527
   Coefficients: [ 6.03256356e-12  1.43433244e-03  5.24687425e-01 -1.39515798e-01
  9.14828173e-02  6.67952058e-03 -1.29365603e-02 -2.71724828e-03
  7.20043194e-03 -6.29695405e-07 -5.94443360e-04  1.34713947e-04
 -9.07824334e-05 -5.20110038e-06  1.19608126e-05  1.47664418e-06
 -7.01758556e-06  9.59816818e-04 -9.84546659e-04  4.30649593e-05
 -1.11192165e-03 -6.18638566e-04  1.72131663e-03  1.54013848e-03
  1.16545055e-04  4.09603184e-05 -5.69845673e-05  1.53151264e-06
  7.87282399e-05  4.54228622e-05  6.87172623e-06 -9.07412922e-06
 -2.33930175e-05  5.29126571e-06 -1.77465790e-05  1.83319418e-05
  2.87277305e-06 -5.34396835e-05 -4.99874977e-06 -1.97409491e-06
  2.71297986e-05 -1.64216638e-05  2.02560222e-05  1.50051273e-05
 -1.81105768e-06]
   Score 0.7520277145213436
MAE is 0.003045019209618236
RMSE is 0.004081072619381468
MSE is 1.6655153724665114e-05
R^2: 0.7520277145213436


In [37]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 2
MAE is 0.003491482096378543
RMSE is 0.005424698928939287
MSE is 2.9427358469635043e-05
R^2: 0.5654891052811468
